## Data Scrapper.

The idea of this project is to have something that is able to:
    
1. Given an input of parameters e.g. category_code, brand (electronics, smartphone, xiaomi)
2. be able to fetch images (the first one) on google images
3. And post them to x database.

In [1]:
list_search = ["electronics, smartphone, xiaomi"]

In [2]:
from bs4 import BeautifulSoup
from PIL import Image
import requests
from io import BytesIO
import IPython.display as Disp

In [3]:

def get_google_img(query):
    """
    gets a link to the first google image search result
    :param query: search query string
    :result: url string to first result
    """
    url = 'https://www.google.com/search?hl=jp&q=' + query + '&btnG=Google+Search&tbs=0&safe=off&tbm=isch'
    headers={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}

    html = requests.get(url, headers=headers).text

    soup = BeautifulSoup(html, 'html.parser')
    image = soup.find("img",{"class":"yWs4tf"})
#     print(image)

    if not image:
        return 
    return image['src']


In [4]:
# query = input("search term\n")
print(get_google_img("banana") or "no image found")

https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQkixelwO2Hk4OJw0xh4HT79G9QjL_CRfOCHNXFYouRYYk5gzsJ_sfZ4P5E8g&s


In [5]:

from PIL import Image, ImageDraw

for imgq in list_search:
    print(f"Searching: {imgq}")
    imgsrc = get_google_img(imgq)
    response = requests.get(imgsrc)
    img = Image.open(BytesIO(response.content))
    img.save("a.jpg")
    
    

Searching: electronics, smartphone, xiaomi


## Finding the unique 

Find unique datasets available


In [7]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv('data/2019-Nov.csv',nrows = 1000000)

In [10]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [17]:
# append category_code and brand and then group them by that

df["item_title"] = df["category_code"] + " " + df["brand"]
df.head(1)


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,item_title
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,electronics.smartphone xiaomi


In [49]:
gb = df.groupby("item_title")
# gb["item_title"].count()
listitems = gb["item_title"].head()
listitems.iloc[:10]

0             electronics.smartphone xiaomi
1          appliances.sewing_machine janome
3              appliances.kitchen.washer lg
4             electronics.smartphone xiaomi
5                     computers.notebook hp
6                     computers.notebook hp
9              electronics.smartphone apple
14    appliances.kitchen.dishwasher samsung
18            electronics.smartphone huawei
19                    computers.notebook hp
Name: item_title, dtype: object

## Downloading the first 10 images

In [65]:
def download_images(list_search):
    for imgq in list_search:
        print(f"Searching: {imgq}")
        imgsrc = get_google_img(imgq)
        response = requests.get(imgsrc)
        img = Image.open(BytesIO(response.content))
        name = imgq
        name = name.replace(" ", "-")
        file_dir = "data/images/"
        img.save(f"{file_dir+name}.jpg")
    

In [67]:
download_images(list(listitems)[:10])

Searching: electronics.smartphone xiaomi
Searching: appliances.sewing_machine janome
Searching: appliances.kitchen.washer lg
Searching: electronics.smartphone xiaomi
Searching: computers.notebook hp
Searching: computers.notebook hp
Searching: electronics.smartphone apple
Searching: appliances.kitchen.dishwasher samsung
Searching: electronics.smartphone huawei
Searching: computers.notebook hp


## Uploading the first 10 images to the API

In [69]:
## Try first uploading one image to the ecommerce API

